In [ ]:
# 🛠️ Step 1: Install ConvoKit
!pip install convokit

# 📦 Step 2: Import and Download the Dataset
from convokit import Corpus, download
import pandas as pd
from collections import defaultdict

# Download the CMV dataset
corpus = Corpus(filename=download("winning-args-corpus"))



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.0/205.0 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.

/usr/local/lib/python3.11/dist-packages/convokit/utterance_simulator/unslothUtteranceSimulatorModel.py:2: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
No configuration file found at /root/.convokit/config.yml; writing with contents: 
# Default Backend Parameters
db_host: localhost:27017
data_directory: ~/.convokit/saved-corpora
model_directory: ~/.convokit/saved-models
default_backend: mem
Number of Speakers: 34911
Number of Utterances: 293297
Number of Conversations: 3051


In [ ]:
# 🌳 Step 3: Extract exactly 10 full conversation trees

all_conversations = []

for i, conv in enumerate(corpus.iter_conversations()):
    utterances = {}
    tree = defaultdict(list)

    # Build reply tree
    for utt in conv.iter_utterances():
        utterances[utt.id] = utt
        tree[utt.reply_to].append(utt.id)

    # Recursive function to build tree for a comment
    def build_tree(utt_id):
        return {
            "id": utt_id,
            "speaker": utterances[utt_id].speaker.id,
            "text": utterances[utt_id].text.strip(),
            "children": [build_tree(child_id) for child_id in tree[utt_id]]
        }

    # Start from top-level replies to the OP
    conversation_tree = [build_tree(child_id) for child_id in tree[None]]

    all_conversations.append({
        "conversation_id": conv.id,
        "title": conv.meta.get("op-title", ""),
        "op_text": conv.meta.get("op-text-body", ""),
        "tree": conversation_tree
    })

    if i >= 9:  # ✅ exactly 10 conversations (index 0 to 9)
        break


In [ ]:
# 💾 Step 4: Save the 10 trees to JSON
with open("cmv_10_conversation_trees.json", "w", encoding="utf-8") as f:
    json.dump(all_conversations, f, ensure_ascii=False, indent=2)

print("✅ Saved 10 conversation trees to 'cmv_10_conversation_trees.json'")


✅ Saved 10 conversation trees to 'cmv_10_conversation_trees.json'
